In [1]:
import nltk
nltk.download('stopwords','tokenizers')

[nltk_data] Downloading package stopwords to tokenizers...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [109]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import types
import time
import re
import queue
import string
import os
import glob
from nltk.corpus import stopwords
import nltk
import string
import math

In [110]:
#Strip the words in doc of any characters that are not alphabets
def makeVal(word):
    word = word.strip(string.punctuation)
    regex=re.compile('[^a-zA-Z]')
    regex.sub('',word)
    word = word.split()
    out_word=''
    for letter in word:
        if letter.isalpha():
            out_word = out_word+letter
    return out_word

In [111]:
#return list of words in the file, remove stop words
def getTokens(filename):
    file = open(filename,'r')
    html = file.read()
    doc = BeautifulSoup(html,'lxml').get_text()
    doc = doc.lower()
    doc = doc.split(' ')
    tokens = []
    for word in doc:
        word = makeVal(word)
        if word not in stopwords.words('english') and len(word)>1 and word != 'html':
            tokens.append(word)
    return tokens

In [112]:
def makeFilePath(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [113]:
def getIndex(folder):
    #Create the index storage dictionary
    inv_index = {}
    doc_num = 0
    #Read all the files in the crawled pages folder
    path = folder + '/'
    for filename in glob.glob(os.path.join(path,'*.html')):
        doc_num += 1
        #Create temp dictionary
        d_freq = {}
        #Convert the file to a list of tokens, or words
        tokens = getTokens(filename)
        for word in tokens:
            if word not in d_freq:
                #add the word to the dictionary as a key, with value 0
                d_freq[word]=1
            else:
                #increment the value of the word in dictionary by 1
                d_freq[word]+=1
        #write the values of d_temp to inv_index
        for word in d_freq:
            value = [doc_num,d_freq[word]]
            if word not in inv_index:
                inv_index[word] = []
                inv_index[word].append(value)
            else:
                inv_index[word].append(value)
    print('Frequency index created!')
    return inv_index

In [114]:
#Get ocurrence based partial index
def getPartIndex(inv_index):
    temp = inv_index.copy()
    for word in inv_index:
        temp[word] = []
        for record in inv_index[word]:
            temp[word].append(record[0])
    return temp

In [115]:
#Returns the intersect of the terms of inv_lists
def Intersect(inv_lists):
    sect_terms = inv_lists[0]
    for term in inv_lists:
        sect_terms = set(sect_terms).intersection(term)
    return sect_terms

In [116]:
#Calculates the support of every itemset
def GetSup(itemset,par_index,sup_vals,num_docs):
    if tuple(itemset) in d_sup_vals:
        return d_sup_vals[tuple(itemset)]
    inv_lists = []
    for term in itemset:
        inv_lists.append(par_index[term])
    else:
        sup = len(Intersect(inv_lists))/num_docs
        d_sup_vals[tuple(itemset)] = sup
    return sup

In [117]:
#Determine if two sets need to be joined
def NeedJoin(set1,set2):
    for i in range(0,len(set1)-1):
        if set1[i] != set2[i]:
            return False
    return True

In [118]:
#Joins two sets
def Join(set1,set2):
    set3 = list(set1)
    set3.append(set2[-1])
    return set3

In [119]:
#Creates an empty list to fill in with frequent itemsets
def getFreqSets(num_terms):
    freq_itemsets = []
    for i in range(0,num_terms+1):
        temp = []
        freq_itemsets.append(temp)
    return freq_itemsets

In [120]:
#Saves the rules to a text file
def SaveRules(rules,num_rules):
    file = open('Analysis report.txt','w')
    file.write('Number of rules generated: '+str(num_rules)+'\n')
    for rule in rules:
        file.write(str(rule[0])+' => '+str(rule[1])+'; support='+str(rule[2])+', confidence='+str(rule[3])+'\n')
    file.close()

In [121]:
#Count the number of crawled documents
def getNumDocs(par_index,path):
    num_docs = 0
    for filename in glob.glob(os.path.join(path,'*.html')):
        num_docs += 1
    return num_docs

In [122]:
def getTopTerms(terms):
    temp_terms = sorted(terms, key=lambda x:x[1],reverse=True)
    temp_terms = temp_terms[0:20]
    top_terms = []
    for term in temp_terms:
        top_terms.append(term[0])
    return top_terms

In [123]:
#Load the related terms to be searched
def getRelTerms(inv_index,num_docs):
    d_temp = {}
    temp_terms = []
    rel_terms = []
    for word in inv_index:
        total_freq = 0
        for term in inv_index[word]:
            total_freq += term[1]
        term_value = num_docs/len(inv_index[word])
        idf = math.log10(term_value)
        temp_terms.append([word,idf])
    rel_terms = getTopTerms(temp_terms)
    print(temp_terms)
    print(rel_terms)
    return rel_terms

In [124]:
path = 'Crawled_Pages'+'/' #Insert folder name here
inv_index = getIndex(path)
par_index = getPartIndex(inv_index)
num_docs = getNumDocs(par_index,path)
rel_terms = getRelTerms(inv_index,num_docs)
min_sup = 0.4
min_conf = 0.5
d_sup_vals = {}
num_terms = len(rel_terms)
freq_itemsets = getFreqSets(num_terms)
#Collection of frequent 1-itemsets
for term in rel_terms:
    itemset = [term]
    if GetSup(itemset,par_index,d_sup_vals,num_docs)>min_sup:
        freq_itemsets[1].append(itemset)

Frequency index created!
[['streatham', 2.615950051656401], ['marriages', 2.218010042984363], ['gang', 2.218010042984363], ['draw', 0.7964061161145324], ['lanteeeleanora', 2.9169800473203824], ['veracity', 2.9169800473203824], ['seddon', 2.43985879260072], ['aylett', 2.9169800473203824], ['unspecified', 2.31492005599242], ['privatised', 2.31492005599242], ['rankunited', 1.7128600646644574], ['declined', 1.1245883578221283], ['hardballtalk', 2.9169800473203824], ['zulch', 2.9169800473203824], ['oriental', 2.0718820073061255], ['crucial', 1.235738809944795], ['cricinfo', 0.5459121850486459], ['compress', 2.615950051656401], ['everybody', 2.31492005599242], ['jani', 2.31492005599242], ['eaters', 2.9169800473203824], ['janowski', 2.9169800473203824], ['ambitiously', 2.9169800473203824], ['footy', 1.6617075422170762], ['jharkhand', 2.43985879260072], ['heaps', 2.9169800473203824], ['updatingwikipedia', 2.615950051656401], ['andreiandrei', 2.9169800473203824], ['indecent', 2.615950051656401]

In [125]:
#Find frequent 2 to K-itemsets
for k in range(2,num_terms+1):
    prev_itemsets = freq_itemsets[k-1]
    n = len(prev_itemsets)
    for i in range(1,n-1):
        for j in range(i+1,n):
            set1 = prev_itemsets[i]
            set2 = prev_itemsets[j]
            if NeedJoin(set1,set2):
                set3 = Join(set1,set2)
                if GetSup(set3,par_index,d_sup_vals,num_docs)>min_sup:
                    freq_itemsets[k].append(set3)
    if len(freq_itemsets) == 1:
        break

In [126]:
#Generate the Association Rules
rules = []
for k in range(2,num_terms+1):
    if not freq_itemsets:
        break
    for itemset in freq_itemsets[k]:
        for i in range(0,k):
            itemset2 = list(itemset)
            itemset2.remove(itemset2[i])
            sup1 = GetSup(itemset2,par_index,d_sup_vals,num_docs)
            sup2 = GetSup(itemset,par_index,d_sup_vals,num_docs)
            conf = sup2/sup1
            if conf>min_conf:
                rule = []
                rule.append(itemset2)
                rule.append(itemset[i])
                rule.append(sup2)
                rule.append(conf)
                rules.append(rule)
num_rules = len(rules)
SaveRules(rules,num_rules)